## helioMOPS (MOPS-TNG)

We need a good and memorable name. Not MOPS -- this is fundamentally different. Optimal if it includes the words Cadence-independent, Range, Cluster (or Associate), and Shift (or Phase). Some ideas:

1. Range, Shift, Cluster and Link Scheme (RaSCaLS)
1. Cadence Independent Linking Algorithm (CILiA, or CILA)
1. Moving Object Shifting And Identification by Clustering (MOSAIC)
1. Targeted Ranging for Asteroid Wide-area Linking (TRAWL)
1. Just call it "trawl", visualizing how we're casting a wide net to trawl the Solar System along test orbits, catching objects that are nearby.
1. Cadence Independent Asteroid Linking Scheme (CIALIS... unfortunately, Eli Lilly (and others) may object :) )
1. ...?

### Mission
To develop an algorithm, applicable to LSST-scale surveys, capable of associating observations of the same (hitherto unknown) asteroids, indepentent of their spacing in time (cadence).

It should approach the ideal that if there are sufficient observations available to compute an orbit (no matter their spacing in time), the object should be findable.

### Motivation

The current implementation of the LSST Moving Object Processing System (MOPS) requires the LSST cadence to deliver at least two visits a night to a single field. Two visits to a single field allows for the creation of tracklets: sky-plane motion vectors in the reference frame of the observer that constrain the direction and velocity of a potential moving object. A minimum of three pairs of visits (three tracklets) taken in a ~two week period are necessary for LSST to deliver its Solar System science data products. Objects that do not follow this pattern will not be successfully linked (e.g., as a particularly bad example, an object with 14 observations over 14 nights, yet with no pairs in a single night, will not be linked).

It would be useful to improve on the existing MOPS algorithm for two reasons:

* Jones et al. (2017) show that there are NEOs that are observed in the LSST baseline observing strategy that are not detected by tracklet-based MOPS. Yet these have _sufficient observations_ to solve for the orbit _if one were able to link them_. Having an algorithm not strongly constrained by cadence could potentially increase the NEO yields by 5 to 10 percentage points.

* Having two visits a night (and in a $\lesssim 60$ minute interval) is not generally required by the other three science drivers. Doing away with this requirement would liberate the cadence optimization process, potentially increasing science yields in other areas.


### The Idea

#### The Algorithm: Shift-and-Cluster

Linking the observations together means identifying point source or trailed detections recorded at various times as being the observations of the same object moving through the Solar System on an (approximately) heliocentric orbit. 

This is challenging for a few reasons:
* There are many objects moving on similar trajectories, and given a set of observations taken at arbitrary times it is not obvious which asteroid from (say) the image A corresponds to which from image B, etc. Note that this becomes possible if the objects are observed in quick succession (so they move less than the typical on-sky distance between two asteroids); this leads to the existing tracklet algorithm which we're trying to improve on.
* The situation is further confused by _false positive detections_ -- artefacts that masquerade as observations, yet arise due to instrumental issues, software impefections, or even photon noise.

If one _knew_ the exact orbit of an object, the linkage would be relatively trivial: simply compute the predicted location of the object and any detections found at those predicted positons are very likely to be the observations of the object at hand. However, as we don't know the exact orbit, this insight does not appear to be of much practical use.

But let us look at this from a different perspective. Given a known orbit, we can also link the observations by computing and undoing the _shifts_ (e.g., $\Delta \alpha$, $\Delta \delta$) of second and subsequent observation relative to the first one. This will _shift back_ all observation to the coordinate frame of the first one, forming a _cluster_ of observations (dispersed only by the observational error) at the location of the initial observation. We can then detect this cluster, and link the observations.

Thinking along these lines leads us to a proposal for a potential cadence independent asteroid linking scheme:

* The first key insight is that, if we allow for the cluster to be broader than the observational error and not exactly at the location of the first observation, it's not necessary to know the _exact_ orbit: a _guess_ (a _test orbit_) will be sufficient to _cluster_ together observations of all asteroids in a bundle of orbits around it. Shifting back the observations of objects on nearby orbits will form clusters of detections detectable above the background of false positives and observations of objects not close to the test orbit. This still leaves a computationally formidable task of densly covering the 4D manifold of allowed orbits (for every small region of the sky) with a sufficient number of test orbits to tease out all possible linkages.

* The second insight is that the shift-and-cluster operation should not be performed in geo- (or topo-) centric frame, but in the _heliocentric_ frame. Viewed from the Sun, observations of an object form a great circle. The motion of one object relative to another on an adjacent orbits will (approximately) form an ellipse over the orbital period $T_{\rm orb}$. (n.b.: note some analogy with the guiding center approximation of Galactic dynamics; also with Bernstein's treatment of KBOs). More importantly, over time spans $T << T_{\rm orb}$, the relative motion _will be linear_. Therefore, if we perform the shift in the heliocentric frame, the clusters be spread along _lines_ in (heliocentric) $(l, b, T_{\rm obs})$ ecliptic coordinates. This _linear structure_ allows us to detect the clusters even when they'd be too diluted for detection by a simple density-of-points criterion, thus reducing the number of test orbits needed to exhaustively cover the 4D manifold.

This approach is similar in many ways to the well known shift-and-stack methods applied at a pixel level to search for extremely faint KBOs. In analogy with shift-and-stack, we term this general idea 'shift-and-cluster'.

TODO: Plug in some of old Jes' figures illustrating the method.

#### Proposed approach

Let us understand the information we have at our disposal for a single observation (detection): it will at the very least provide us with information as to the location of the detection, the time, the magnitude (in a specific filter), and the associated signal to noise ratio. 

$$ \text{DIASource} : [(t_i, \alpha_i, \delta_i), m_{f, i}, snr_{f, i}] $$

Note that what is missing is the distance to the object (frequently referred to as the _range_, $\Delta$); we only know the direction in which the object appears (two out of three 3D coordinates).

In addition to these parameters, we know the location of the observer (the telescope) relative to the Sun:

$$ \vec{U_{obs}} (t_i) = (x_{obs}(t_i), y_{obs}(t_i), z_{obs}(t_i)) $$

where we assume the reference axes are tied to J2000-Epoch ecliptic coordinate system. In this case, we know all three coordinates.

In order to try to link any DIASource detections using shift-and-cluster we need to make a guess for ($\Delta, \dot \Delta$, $\dot \alpha$, $\dot \delta$) and shift any observations along the test orbit. Instead of the range to the observer, it is better to instead use the heliocentric distance, $r$. This is a simple change of variables, but it has the benefit that the range of plausible $\dot r$ is much smaller than the range of plausible $\dot \Delta$ (e.g., $\dot \Delta$ reaches velocities similar to orbital velocities, while $\dot r$ is typically much smaller). In fact, a guess of $\dot \Delta = 0$ may be sufficient to recover many objects.

Once we assume some reasonable heliocentric distance we can transform equatorial coordinates $(\alpha_i, \delta_i)$ to cartesian coordinates $(x_i, y_i, z_i)$, as well as their derivatives. From there we can perform a procedure similar to the work of Bernstein & Khushalani and transform to tangent plane coordinates from the reference point of the Sun. These tangent plane coordinates take the form $(\theta_x, \theta_y)$. So the coordinates in which we sample the 4D space of orbits are $(r, \dot r, \dot \theta_x, \dot \theta_y)$ and we can start by assuming $\dot r = 0$.

[[[ In the ideal case, each individial asteroid will generate many different DIASources. Within the span of a few days, or in other words, within the span of a few visits to the same or nearby fields we expect any small body to generate DIASources. If for any given field we make a guess at some reasonable heliocentric distance we can create a set of tangent plane projections. ]]] -- I'd think of this as us having chosen a test orbit that crosses the initial field, and then trace where it crossess future observations. Where there's an intersection, we shift those observations along the test orbit back to $T_{\rm initial}$.

[[[ In this tangent plane projection in the frame of the Sun we can phase-fold nearby field visits assuming some velocity (in principle this can be done at the per field level or at the per DIASource level). If our estimate of the velocity and heliocentric distance was correct we would anticipate that in the phase-folded image a true object would appear as a clustered set of points. ]]] -- note that when we say "field" here, that's larger than the LSST field; think of it as a ~10deg wide circle on the sky around the test orbit. We should find a better name; a "cell"?

In the likely event that we guessed the distance and velocity incorrectly for a subset of DIASources tied to a population of asteroids we would expect to see some linear features in the phase-folded image. Using a 3D Hough transform these features can be extracted and we can then calculate the most probable heliocentric distance and establish potential linkages.

(To test: if we assume some rotational direction about the Sun, we could check if we overestimated or underestimated the distance by looking at the direction that these trails take in the phase-folded image.)

### Proposed Initial Experiment

1) Grab the latest MPCORB.dat. Compute the ephemerides for one date, for the entire catalog.

2) Select all objects in a ~60 degree around some point near the opposition and centered on the eclipic (for simplicity). Generate ~30 days of ephemeris at 1 day intervals for those objects.

3) For simplicity for this initial experiment, pick one "typical" asteroid near the center of the ~60 deg sample and adopt its orbit as the test orbit. Try to find one with low $e \approx 0$. Going forward, we'd pick a heliocentric distance, $\dot r$, and the inclination + tangential velocity.

4) Compute the ephemeris of the test orbit object over the next 30 days.

5) Transform the positions of all objects to heliocentric coordinates, assuming their heliocentric distance of all observed objects is the same as that of the test orbit object. Then transform these to the tangent plane coordinates with the test-orbit object being at the center of the tangent plane (for each day), and the $y=0$ line being the plane of the test orbit. This will be a bit of vector/matrix math.

6) Phase-fold the objects (this becomes trivial after 5 -- they're already phase folded as the test object is at $x=y=0$).

7) Look for any clustered points (by eye at first). Confirm Jes' observation that the objects on nearby orbits show as linear in space & time. See how valid is that approximation.

8) Peform Hough transform to identify any linear features. See which objects we catch, and estimate the size of the bundle of orbits that we're effectivelly exploring with this one orbit.

9) Repeat 3-8 as necessary